In [1]:
%pip install mlflow
%pip install pytest
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [2]:
#%pip install tensorflow
#%pip install tensorflow.keras.applications

import mlflow, time, pickle, sys
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
#from tensorflow.keras.applications import VGG16

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

from sklearn.metrics import f1_score  # Import f1_score

sys.path.append('/home/w4z3/Git/MLOPS-cifar10')




In [18]:
%pip install src==0.0.7
#sys.path.append('C:\Users\PC\Desktop\Barcelona\MLOPS\GITHUB\MLOPS-cifar10')
from src import TRAIN_DATA_DIR, TEST_DATA_DIR, MODELS_DIR

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3732572943.py, line 2)

In [5]:
%pip install codecarbon
from codecarbon import track_emissions, EmissionsTracker

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Define the desired input size
input_size = (75, 75)

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Apply InceptionV3 preprocessing
    zoom_range=0.2,
    horizontal_flip=True
)

batch_size = 32  # Adjust as needed

train_generator = datagen.flow_from_directory(
    #"../data/test/",
    TRAIN_DATA_DIR,
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    #"../data/test/",
    TEST_DATA_DIR,  
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [9]:
with EmissionsTracker() as tracker:

    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(75, 75, 3))
    for layer in base_model.layers:
        layer.trainable = False

    x = Flatten()(base_model.output)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(10, activation='softmax')(x)

[codecarbon INFO @ 21:36:33] [setup] RAM Tracking...
[codecarbon INFO @ 21:36:33] [setup] GPU Tracking...
[codecarbon INFO @ 21:36:33] No GPU found.
[codecarbon INFO @ 21:36:33] [setup] CPU Tracking...
[codecarbon WARNING @ 21:36:33] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 21:36:38] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz
[codecarbon INFO @ 21:36:38] >>> Tracker's metadata:
[codecarbon INFO @ 21:36:38]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 21:36:38]   Python version: 3.10.11
[codecarbon INFO @ 21:36:38]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 21:36:38]   Available RAM : 15.900 GB
[codecarbon INFO @ 21:36:38]   CPU count: 8
[codecarbon INFO @ 21:36:38]   CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz
[codecarbon INFO @ 21:36:38]   GPU count: None
[codecarbon INFO @ 21:36:38]   GPU model: None


27795456/87910968 [========>.....................] - ETA: 16s

[codecarbon INFO @ 21:36:53] Energy consumed for RAM : 0.000025 kWh. RAM Power : 5.962331771850587 W


27844608/87910968 [========>.....................] - ETA: 16s

[codecarbon INFO @ 21:36:53] Energy consumed for all CPUs : 0.000031 kWh. Total CPU Power : 7.5 W
[codecarbon INFO @ 21:36:53] 0.000056 kWh of electricity used since the beginning.


85778432/87910968 [============================>.] - ETA: 0s

[codecarbon INFO @ 21:37:08] Energy consumed for RAM : 0.000050 kWh. RAM Power : 5.962331771850587 W
[codecarbon INFO @ 21:37:08] Energy consumed for all CPUs : 0.000063 kWh. Total CPU Power : 7.5 W
[codecarbon INFO @ 21:37:08] 0.000112 kWh of electricity used since the beginning.


87910968/87910968 [==============================] - 23s 0us/step


[codecarbon INFO @ 21:37:10] Energy consumed for RAM : 0.000054 kWh. RAM Power : 5.962331771850587 W
[codecarbon INFO @ 21:37:10] Energy consumed for all CPUs : 0.000068 kWh. Total CPU Power : 7.5 W
[codecarbon INFO @ 21:37:10] 0.000122 kWh of electricity used since the beginning.


In [10]:

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

[codecarbon INFO @ 21:39:20] [setup] RAM Tracking...
[codecarbon INFO @ 21:39:20] [setup] GPU Tracking...
[codecarbon INFO @ 21:39:20] No GPU found.
[codecarbon INFO @ 21:39:20] [setup] CPU Tracking...
[codecarbon WARNING @ 21:39:20] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 21:39:24] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz
[codecarbon INFO @ 21:39:24] >>> Tracker's metadata:
[codecarbon INFO @ 21:39:24]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 21:39:24]   Python version: 3.10.11
[codecarbon INFO @ 21:39:24]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 21:39:24]   Available RAM : 15.900 GB
[codecarbon INFO @ 21:39:24]   CPU count: 8
[codecarbon INFO @ 21:39:24]   CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz
[codecarbon INFO @ 21:39:24]   GPU count: None
[codecarbon INFO @ 21:39:24]   GPU model: None
[codecarbon INFO @ 21:39:25] Energy consumed for RAM : 0.000000 kWh. RAM Pow

In [14]:
num_epochs = 10  # Adjust as needed
batch_size = 32  # Adjust as needed

In [16]:
# mlflow.set_experiment("cifar10")
# mlflow.sklearn.autolog(log_model_signatures=False, log_datasets=False)

# with mlflow.start_run():
#   mlflow.log_params({
#     "model_name": "imagenet-cifar10" ,
#     "dataset_name": "CIFAR-10 Test" ,
#     "num_epochs": num_epochs ,
#     "batch_size": batch_size
#   })

with EmissionsTracker() as tracker:
  model.fit(train_generator, epochs=num_epochs, batch_size=batch_size)
  #model.save('../models/cifar10.h5')#MODELS_DIR/'cifar10.h5')
  model.save(MODELS_DIR/'cifar10.h5')
  #loaded_model = tf.keras.models.load_model('your_model.h5')

evaluation = model.evaluate(test_generator)

loss = evaluation[0]
accuracy = evaluation[1]
# mlflow.log_metric("evaluation_loss", loss)
# mlflow.log_metric("evaluation_accuracy", accuracy)

# Calculate F1 score
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred = y_pred.argmax(axis=-1)  # Convert predicted probabilities to class labels
f1score = f1_score(y_true, y_pred, average='weighted')
# mlflow.log_metric("evaluation_f1score", f1score)

# Log model
# with open(MODELS_DIR / "cifar10.pkl", "wb") as pickle_file:
#     pickle.dump(model, pickle_file)
  

[codecarbon INFO @ 22:42:13] [setup] RAM Tracking...
[codecarbon INFO @ 22:42:13] [setup] GPU Tracking...
[codecarbon INFO @ 22:42:13] No GPU found.
[codecarbon INFO @ 22:42:13] [setup] CPU Tracking...
[codecarbon WARNING @ 22:42:13] No CPU tracking mode found. Falling back on CPU constant mode.


[codecarbon INFO @ 22:42:18] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz
[codecarbon INFO @ 22:42:18] >>> Tracker's metadata:
[codecarbon INFO @ 22:42:18]   Platform system: Windows-10-10.0.19045-SP0
[codecarbon INFO @ 22:42:18]   Python version: 3.10.11
[codecarbon INFO @ 22:42:18]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 22:42:18]   Available RAM : 15.900 GB
[codecarbon INFO @ 22:42:18]   CPU count: 8
[codecarbon INFO @ 22:42:18]   CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz
[codecarbon INFO @ 22:42:18]   GPU count: None
[codecarbon INFO @ 22:42:18]   GPU model: None


Epoch 1/10
 38/313 [==>...........................] - ETA: 1:42 - loss: 0.8362 - accuracy: 0.7072

[codecarbon INFO @ 22:42:34] Energy consumed for RAM : 0.000025 kWh. RAM Power : 5.962331771850587 W
[codecarbon INFO @ 22:42:34] Energy consumed for all CPUs : 0.000031 kWh. Total CPU Power : 7.5 W
[codecarbon INFO @ 22:42:34] 0.000056 kWh of electricity used since the beginning.


 80/313 [======>.......................] - ETA: 1:24 - loss: 0.8387 - accuracy: 0.7094

...

296/313 [===========================>..] - ETA: 5s - loss: 0.7324 - accuracy: 0.7424

[codecarbon INFO @ 22:59:50] Energy consumed for RAM : 0.001737 kWh. RAM Power : 5.962331771850587 W
[codecarbon INFO @ 22:59:50] Energy consumed for all CPUs : 0.002189 kWh. Total CPU Power : 7.5 W
[codecarbon INFO @ 22:59:50] 0.003927 kWh of electricity used since the beginning.


313/313 [==============================] - 94s 299ms/step - loss: 0.7312 - accuracy: 0.7434


C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
[codecarbon INFO @ 22:59:57] Energy consumed for RAM : 0.001749 kWh. RAM Power : 5.962331771850587 W
[codecarbon INFO @ 22:59:57] Energy consumed for all CPUs : 0.002203 kWh. Total CPU Power : 7.5 W
[codecarbon INFO @ 22:59:57] 0.003952 kWh of electricity used since the beginning.


313/313 [==============================] - 98s 303ms/step
